In [1]:
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from tqdm import tqdm
from tqdm_joblib import tqdm_joblib
from joblib import Parallel, delayed

g:\anaconda3\envs\porto\lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
# stopwords
stopwords_id = set(stopwords.words('indonesian'))

# stemmer
stemmer = StemmerFactory().create_stemmer()

# fungsi preprocess
def preprocess(text):
    text = text.lower()
    tokens = [w for w in text.split() if w not in stopwords_id]
    text = " ".join(tokens)
    return stemmer.stem(text)

In [3]:
# load dataset
df = pd.read_csv("News.csv")

# list content + tqdm untuk progress bar
contents = df['content'].fillna('').astype(str).tolist()

# results = Parallel(n_jobs=-1, batch_size=5000)(
#     delayed(preprocess)(doc) for doc in tqdm(contents, desc="Preprocessing")
# )

with tqdm_joblib(tqdm(total=len(contents), desc="Preprocessing")):
    results = Parallel(n_jobs=-1)(
        delayed(preprocess)(doc) for doc in contents
    )

df['content_clean'] = results

Preprocessing:   0%|          | 0/14343 [00:00<?, ?it/s]

  0%|          | 0/14343 [00:00<?, ?it/s]

In [4]:
output_file = "News_processed.csv"
df.to_csv(output_file, index=False)
print(f"Hasil preprocessing berhasil disimpan ke: {output_file}")

Hasil preprocessing berhasil disimpan ke: News_processed.csv
